<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-2-validate-classification-problems/LS_DS_242_Validate_classification_problems_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working on Synthetic Financial Dataset For Fraud Detection data from Kaggle

Source: https://www.kaggle.com/ntnu-testimon/paysim1

In [1]:
!pip install kaggle

In [2]:
# Mount the drive to download the data from Kaggle
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive


In [3]:
!kaggle datasets download -d ntnu-testimon/paysim1

paysim1.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip paysim1.zip

Archive:  paysim1.zip
replace PS_20174392719_1491204439457_log.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
# Generic imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='once')

In [0]:
frauddf = pd.read_csv('PS_20174392719_1491204439457_log.csv')

In [7]:
frauddf.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [8]:
frauddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
step              int64
type              object
amount            float64
nameOrig          object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest          object
oldbalanceDest    float64
newbalanceDest    float64
isFraud           int64
isFlaggedFraud    int64
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [9]:
frauddf.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [10]:
# Identify the unique values and type for each column
for col in frauddf.columns:
  print(col+'\t'+str(frauddf[col].dtypes))
  print(len(frauddf[col].unique()))

step	int64
743
type	object
5
amount	float64
5316900
nameOrig	object
6353307
oldbalanceOrg	float64
1845844
newbalanceOrig	float64
2682586
nameDest	object
2722362
oldbalanceDest	float64
3614697
newbalanceDest	float64
3555499
isFraud	int64
2
isFlaggedFraud	int64
2


In [0]:
# nameOrig and nameDest columns have information on users which is not needed
# for One Hot Encoding so dropping it.
X = frauddf.drop(columns=['isFraud','nameOrig','nameDest'])
y = frauddf.isFraud

In [12]:
# Method to split the data into train, validation and test sets
from sklearn.model_selection import train_test_split

def train_validation_test_split(
    X, y, train_size=0.8, val_size=0.1, test_size=0.1, 
    random_state=None, shuffle=True):
        
    assert train_size + val_size + test_size == 1
    
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=shuffle)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=val_size/(train_size+val_size), 
        random_state=random_state, shuffle=shuffle)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [0]:
X_train, X_val, X_test, y_train, y_val, y_test \
= train_validation_test_split(X, 
                              y, 
                              train_size=0.8, 
                              val_size=0.1, 
                              test_size=0.1, 
                              random_state=42,
                              shuffle=True)

In [14]:
# Baseline for Fraud detection
y_val.value_counts(normalize=True)

0    0.998669
1    0.001331
Name: isFraud, dtype: float64

In [0]:
import numpy as np

majority_class = y_train.mode()[0]
y_pred = np.full(shape=y_val.shape, fill_value=majority_class)

In [16]:
from sklearn.metrics import accuracy_score, classification_report, \
roc_auc_score, confusion_matrix

accuracy_score(y_val, y_pred)

0.9986687873863282

In [17]:
roc_auc_score(y_val,y_pred)

0.5

In [18]:
confusion_matrix(y_val, y_pred)

array([[635415,      0],
       [   847,      0]])

**Baseline:**

Accuracy - 0.9986687873863282

ROCAUC Score - 0.5

In [19]:
!pip install category_encoders

In [20]:
#Logistic Regression with class_weight=None
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(\
                        ce.OneHotEncoder(cols=['type'],
                                         use_cat_names=True),
                         StandardScaler(),
                         LogisticRegression(solver='lbfgs')
                        )

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)


/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transfor

In [21]:
from sklearn.metrics import accuracy_score, classification_report, \
roc_auc_score, confusion_matrix

accuracy_score(y_val, y_pred)

0.9992015867677152

In [22]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    635415
           1       0.93      0.43      0.59       847

   micro avg       1.00      1.00      1.00    636262
   macro avg       0.97      0.72      0.79    636262
weighted avg       1.00      1.00      1.00    636262



In [23]:
roc_auc_score(y_val,y_pred)

0.7160354246447587

In [24]:
confusion_matrix(y_val, y_pred)

array([[635388,     27],
       [   481,    366]])

**Logistic Regression with class_weight=None:** 

Accuracy - 0.9992015867677152

ROCAUC Score - 0.7160354246447587

In [25]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64

In [26]:
score, score.mean(), score.std()

(array([0.99924952, 0.99922595, 0.9992397 , 0.99921613, 0.99921416,
        0.99922988, 0.99922398, 0.99925345, 0.99923773, 0.99922005]),
 0.9992310557614367,
 1.2918561352239522e-05)

Accuracy score is consistently high and also the std deviation is very low.

**Cross Validation Output:**

0.99924952

0.99922595

0.9992397

0.99921613

0.99921416

0.99922988

0.99922398

0.99925345

0.99923773

0.99922005
        
 ** Cross validation mean:**
 
 0.9992310557614367
 
 ** Cross validation std deviation:**
 
 1.2918561352239522e-05



In [27]:
#Logistic Regression with class_weight=balanced
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(\
                        ce.OneHotEncoder(cols=['type'],
                                         use_cat_names=True),
                         StandardScaler(),
                         LogisticRegression(class_weight='balanced',
                                            solver='lbfgs')
                        )

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [28]:
accuracy_score(y_val, y_pred)

0.9492551810417721

In [29]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97    635415
           1       0.03      0.99      0.05       847

   micro avg       0.95      0.95      0.95    636262
   macro avg       0.51      0.97      0.51    636262
weighted avg       1.00      0.95      0.97    636262



In [30]:
roc_auc_score(y_val,y_pred)

0.9681089186931824

In [31]:
confusion_matrix(y_val, y_pred)

array([[603139,  32276],
       [    11,    836]])

**Logistic Regression with class_weight=balanced:** 

Accuracy - 0.9492551810417721

ROCAUC Score - 0.9681089186931824

In [32]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64

In [33]:
score, score.mean(), score.std()

(array([0.95222687, 0.95235654, 0.95415218, 0.9524862 , 0.95177305,
        0.95111687, 0.95301271, 0.9489675 , 0.95148612, 0.95202237]),
 0.9519600409615444,
 0.001278611958478853)

Accuracy score is relatively low high and also the std deviation low.

**Cross Validation Output:**

0.95222687

0.95235654

0.95415218

0.9524862

0.95177305,

0.95111687

0.95301271

0.9489675

0.95148612

0.95202237
        
 ** Cross validation mean:**
 
 0.9519600409615444
 
 ** Cross validation std deviation:**
 
0.001278611958478853

